In [1]:
import time

import numpy as np
import pandas as pd
from pandas import DataFrame
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
import random

import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig

In [2]:
sensor = pd.read_csv('data/sensor_infomation.csv')
sensor

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
0,715933,5,N,7,37,14974.0,10.33,126.963,33.981839,-118.130679,0.425,ML,4,GREENWOOD,2610,NaN,NaN,NaN
1,715938,5,N,7,37,14974.0,12.18,128.813,34.002541,-118.150997,0.545,ML,4,GASPAR,2021,NaN,NaN,NaN
2,715944,5,N,7,37,14974.0,13.35,129.983,34.013676,-118.166091,0.323,ML,4,FERRIS,2537,NaN,NaN,NaN
3,715947,5,S,7,37,14974.0,13.67,130.240,34.015325,-118.171270,0.495,ML,3,S OF 710,2536,NaN,NaN,NaN
4,716010,5,S,7,37,69088.0,R49.5,165.847,34.368149,-118.558850,0.600,ML,4,S OF WABUSKA,2089,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,774672,110,N,7,37,44000.0,21.7,21.630,34.041951,-118.273107,0.370,ML,3,14TH STREET,2608,NaN,NaN,NaN
146,774728,10,W,7,37,58072.0,47.2,45.700,34.081720,-117.728629,0.335,ML,4,E/O SAN ANTONIO,2129,NaN,NaN,NaN
147,775155,405,N,7,37,NaN,31.3,55.072,34.052632,-118.449662,0.310,ML,6,NO OF SANTA MONICA,2061,NaN,NaN,NaN
148,775610,110,S,7,37,44000.0,23.8,23.730,34.063457,-118.247963,0.705,ML,3,SUNSET,2392,NaN,NaN,NaN


In [3]:
training_flow = pd.read_csv('data/training_flow.csv')
training_speed = pd.read_csv('data/training_speed.csv')
training_flow.shape

(69695, 151)

In [4]:
test_flow = pd.read_csv('data/test_flow.csv')
test_speed = pd.read_csv('data/test_speed.csv')
test_flow.shape

(5471, 151)

In [5]:
training_set_q = torch.from_numpy(training_flow.iloc[:,1:].values).to(torch.float32)
test_set_q = torch.from_numpy(test_flow.iloc[:,1:].values).to(torch.float32)

training_set_v = torch.from_numpy(training_speed.iloc[:,1:].values).to(torch.float32)
test_set_v = torch.from_numpy(test_speed.iloc[:,1:].values).to(torch.float32)

num_sensor = training_set_q.shape[1]
num_sensor

150

In [6]:
training_set_k = training_set_q/training_set_v
test_set_k = test_set_q/test_set_v
training_set_k.shape

torch.Size([69695, 150])

In [7]:
upper_bound_critical_density = 45 * torch.ones(num_sensor) # we assume that the upper bound of critical density is 45 vehicle/mile
lower_bound_critical_density = 30 * torch.ones(num_sensor) # we assume that the upper bound of critical density is 30 vehicle/mile

# Assume that the lower bound of freeflow speed should be larger than the mean value of speed
lower_bound_FFS = training_set_v.quantile(0.7,axis=0)
upper_bound_FFS = torch.max(training_set_v.quantile(0.95,axis=0),lower_bound_FFS+0.1)

lower_bound_mm = 0 * torch.ones(num_sensor)
upper_bound_mm = 15 * torch.ones(num_sensor)

In [8]:
# S3
class MFD(Module):
    
    def __init__(self,):
        super(MFD, self).__init__()
        
        self.v_f = Parameter(lower_bound_FFS+5)
        self.k_c = Parameter(35 * torch.ones(num_sensor))
        self.mm = Parameter(4 * torch.ones(num_sensor))
        
    def forward(self, k):
        
        v_f_ = torch.max(self.v_f,lower_bound_FFS)
        v_f_ = torch.min(v_f_,upper_bound_FFS)
        
        k_c_ = torch.max(self.k_c,lower_bound_critical_density)
        k_c_ = torch.min(k_c_,upper_bound_critical_density)
        
        mm_ = torch.max(self.mm,lower_bound_mm)
        mm_ = torch.min(mm_,upper_bound_mm)
        
        v_estimation = v_f_ / (1 + (k/k_c_) ** mm_) ** (2/mm_)
        
        return v_estimation, v_f_, k_c_, mm_

In [9]:
mfd = MFD()

In [10]:
# Loss function
mse_loss = nn.MSELoss()
# Optimizers
optimizer = torch.optim.Adam(mfd.parameters(), lr=0.001, betas=(0.5, 0.999))
n_epochs = 20000
batch_size = 512
num_sample = training_set_k.shape[0]

In [11]:
loss_training = torch.zeros(200)
loss_test = torch.zeros(200)

def train(n_epochs):
    t0 = time.time()
    for epoch in range(n_epochs):
        
        optimizer.zero_grad()
        
        idx = random.sample(range(0, num_sample),batch_size)
        
        v_estimation,v_f_,k_c_, mm_ = mfd(training_set_k[idx])
        
        loss_target = Variable(torch.Tensor(v_estimation.shape).fill_(1.0), requires_grad=False)
        loss = mse_loss(v_estimation/training_set_v[idx],loss_target)
        
        if (epoch + 1) % 100 == 0:
            
            loss_training[(epoch // 100)] = loss.item()
            
            v_estimation_test = mfd(test_set_k)[0]
            loss_target_test = Variable(torch.Tensor(v_estimation_test.shape).fill_(1.0), requires_grad=False)
            loss_test[(epoch // 100)] = mse_loss(v_estimation_test/test_set_v,loss_target_test)
        
        loss.backward()
        optimizer.step()
        
        print(
            "[Epoch %d/%d] [loss: %f] [time: %f]"
            % (epoch, n_epochs, loss.item(), time.time()-t0)
        )

In [12]:
train(n_epochs = n_epochs)

[Epoch 0/20000] [loss: 0.036782] [time: 0.056696]
[Epoch 1/20000] [loss: 0.038226] [time: 0.067647]
[Epoch 2/20000] [loss: 0.038171] [time: 0.076607]
[Epoch 3/20000] [loss: 0.038501] [time: 0.085620]
[Epoch 4/20000] [loss: 0.036496] [time: 0.094580]
[Epoch 5/20000] [loss: 0.036571] [time: 0.104537]
[Epoch 6/20000] [loss: 0.038193] [time: 0.113496]
[Epoch 7/20000] [loss: 0.038307] [time: 0.123453]
[Epoch 8/20000] [loss: 0.035042] [time: 0.132362]
[Epoch 9/20000] [loss: 0.036841] [time: 0.141322]
[Epoch 10/20000] [loss: 0.037301] [time: 0.150334]
[Epoch 11/20000] [loss: 0.038584] [time: 0.159294]
[Epoch 12/20000] [loss: 0.038571] [time: 0.168254]
[Epoch 13/20000] [loss: 0.036302] [time: 0.177213]
[Epoch 14/20000] [loss: 0.036406] [time: 0.186174]
[Epoch 15/20000] [loss: 0.038567] [time: 0.194140]
[Epoch 16/20000] [loss: 0.035526] [time: 0.203100]
[Epoch 17/20000] [loss: 0.035642] [time: 0.211065]
[Epoch 18/20000] [loss: 0.038883] [time: 0.220025]
[Epoch 19/20000] [loss: 0.038575] [time: 

[Epoch 165/20000] [loss: 0.034630] [time: 1.561116]
[Epoch 166/20000] [loss: 0.034888] [time: 1.570080]
[Epoch 167/20000] [loss: 0.033382] [time: 1.579038]
[Epoch 168/20000] [loss: 0.032894] [time: 1.587949]
[Epoch 169/20000] [loss: 0.035330] [time: 1.596909]
[Epoch 170/20000] [loss: 0.035349] [time: 1.605870]
[Epoch 171/20000] [loss: 0.034120] [time: 1.613835]
[Epoch 172/20000] [loss: 0.034759] [time: 1.622848]
[Epoch 173/20000] [loss: 0.039832] [time: 1.631754]
[Epoch 174/20000] [loss: 0.032799] [time: 1.640713]
[Epoch 175/20000] [loss: 0.035266] [time: 1.649673]
[Epoch 176/20000] [loss: 0.033265] [time: 1.658633]
[Epoch 177/20000] [loss: 0.035097] [time: 1.667593]
[Epoch 178/20000] [loss: 0.031568] [time: 1.676553]
[Epoch 179/20000] [loss: 0.031510] [time: 1.685514]
[Epoch 180/20000] [loss: 0.032601] [time: 1.693482]
[Epoch 181/20000] [loss: 0.036859] [time: 1.702442]
[Epoch 182/20000] [loss: 0.034511] [time: 1.710407]
[Epoch 183/20000] [loss: 0.033627] [time: 1.718372]
[Epoch 184/2

[Epoch 336/20000] [loss: 0.035440] [time: 3.099342]
[Epoch 337/20000] [loss: 0.033343] [time: 3.108302]
[Epoch 338/20000] [loss: 0.033097] [time: 3.117262]
[Epoch 339/20000] [loss: 0.026776] [time: 3.127217]
[Epoch 340/20000] [loss: 0.029913] [time: 3.136189]
[Epoch 341/20000] [loss: 0.030245] [time: 3.145137]
[Epoch 342/20000] [loss: 0.031931] [time: 3.154098]
[Epoch 343/20000] [loss: 0.032258] [time: 3.163010]
[Epoch 344/20000] [loss: 0.030167] [time: 3.171027]
[Epoch 345/20000] [loss: 0.030202] [time: 3.179996]
[Epoch 346/20000] [loss: 0.035098] [time: 3.188944]
[Epoch 347/20000] [loss: 0.031093] [time: 3.196912]
[Epoch 348/20000] [loss: 0.033895] [time: 3.205870]
[Epoch 349/20000] [loss: 0.031209] [time: 3.213835]
[Epoch 350/20000] [loss: 0.031950] [time: 3.222797]
[Epoch 351/20000] [loss: 0.032056] [time: 3.232753]
[Epoch 352/20000] [loss: 0.030248] [time: 3.241713]
[Epoch 353/20000] [loss: 0.033854] [time: 3.249678]
[Epoch 354/20000] [loss: 0.031698] [time: 3.258641]
[Epoch 355/2

[Epoch 513/20000] [loss: 0.029429] [time: 4.676337]
[Epoch 514/20000] [loss: 0.031354] [time: 4.685297]
[Epoch 515/20000] [loss: 0.031012] [time: 4.693265]
[Epoch 516/20000] [loss: 0.028215] [time: 4.702225]
[Epoch 517/20000] [loss: 0.030115] [time: 4.711185]
[Epoch 518/20000] [loss: 0.031485] [time: 4.720145]
[Epoch 519/20000] [loss: 0.028812] [time: 4.729106]
[Epoch 520/20000] [loss: 0.031493] [time: 4.738129]
[Epoch 521/20000] [loss: 0.029851] [time: 4.746085]
[Epoch 522/20000] [loss: 0.030040] [time: 4.755046]
[Epoch 523/20000] [loss: 0.029738] [time: 4.763010]
[Epoch 524/20000] [loss: 0.026494] [time: 4.771971]
[Epoch 525/20000] [loss: 0.026855] [time: 4.780931]
[Epoch 526/20000] [loss: 0.029340] [time: 4.788896]
[Epoch 527/20000] [loss: 0.026779] [time: 4.797855]
[Epoch 528/20000] [loss: 0.030314] [time: 4.806824]
[Epoch 529/20000] [loss: 0.028740] [time: 4.815777]
[Epoch 530/20000] [loss: 0.028590] [time: 4.824737]
[Epoch 531/20000] [loss: 0.028167] [time: 4.833699]
[Epoch 532/2

[Epoch 689/20000] [loss: 0.029724] [time: 6.240501]
[Epoch 690/20000] [loss: 0.027534] [time: 6.249461]
[Epoch 691/20000] [loss: 0.025070] [time: 6.258421]
[Epoch 692/20000] [loss: 0.026580] [time: 6.266388]
[Epoch 693/20000] [loss: 0.027413] [time: 6.275348]
[Epoch 694/20000] [loss: 0.024842] [time: 6.285303]
[Epoch 695/20000] [loss: 0.027820] [time: 6.295262]
[Epoch 696/20000] [loss: 0.025263] [time: 6.304219]
[Epoch 697/20000] [loss: 0.027720] [time: 6.313180]
[Epoch 698/20000] [loss: 0.026734] [time: 6.322140]
[Epoch 699/20000] [loss: 0.025792] [time: 6.340062]
[Epoch 700/20000] [loss: 0.029180] [time: 6.349023]
[Epoch 701/20000] [loss: 0.026832] [time: 6.356934]
[Epoch 702/20000] [loss: 0.026772] [time: 6.365948]
[Epoch 703/20000] [loss: 0.025866] [time: 6.373914]
[Epoch 704/20000] [loss: 0.026697] [time: 6.382875]
[Epoch 705/20000] [loss: 0.026327] [time: 6.391840]
[Epoch 706/20000] [loss: 0.027147] [time: 6.400793]
[Epoch 707/20000] [loss: 0.026884] [time: 6.408758]
[Epoch 708/2

...

[Epoch 19373/20000] [loss: 0.014160] [time: 172.483495]
[Epoch 19374/20000] [loss: 0.014831] [time: 172.492511]
[Epoch 19375/20000] [loss: 0.014364] [time: 172.501416]
[Epoch 19376/20000] [loss: 0.015677] [time: 172.510427]
[Epoch 19377/20000] [loss: 0.014046] [time: 172.518392]
[Epoch 19378/20000] [loss: 0.015884] [time: 172.527353]
[Epoch 19379/20000] [loss: 0.015616] [time: 172.536312]
[Epoch 19380/20000] [loss: 0.016684] [time: 172.545275]
[Epoch 19381/20000] [loss: 0.015472] [time: 172.554232]
[Epoch 19382/20000] [loss: 0.014588] [time: 172.563195]
[Epoch 19383/20000] [loss: 0.017608] [time: 172.571108]
[Epoch 19384/20000] [loss: 0.015757] [time: 172.580069]
[Epoch 19385/20000] [loss: 0.015707] [time: 172.588034]
[Epoch 19386/20000] [loss: 0.015476] [time: 172.596049]
[Epoch 19387/20000] [loss: 0.015214] [time: 172.605018]
[Epoch 19388/20000] [loss: 0.014704] [time: 172.612973]
[Epoch 19389/20000] [loss: 0.016201] [time: 172.621939]
[Epoch 19390/20000] [loss: 0.015698] [time: 172.

[Epoch 19525/20000] [loss: 0.015233] [time: 173.842506]
[Epoch 19526/20000] [loss: 0.014404] [time: 173.850471]
[Epoch 19527/20000] [loss: 0.015666] [time: 173.859432]
[Epoch 19528/20000] [loss: 0.016479] [time: 173.867397]
[Epoch 19529/20000] [loss: 0.015870] [time: 173.876409]
[Epoch 19530/20000] [loss: 0.014046] [time: 173.885368]
[Epoch 19531/20000] [loss: 0.017235] [time: 173.894328]
[Epoch 19532/20000] [loss: 0.016388] [time: 173.902292]
[Epoch 19533/20000] [loss: 0.015042] [time: 173.911255]
[Epoch 19534/20000] [loss: 0.014944] [time: 173.920216]
[Epoch 19535/20000] [loss: 0.014730] [time: 173.929125]
[Epoch 19536/20000] [loss: 0.015707] [time: 173.937139]
[Epoch 19537/20000] [loss: 0.016255] [time: 173.946101]
[Epoch 19538/20000] [loss: 0.016191] [time: 173.954066]
[Epoch 19539/20000] [loss: 0.017057] [time: 173.962032]
[Epoch 19540/20000] [loss: 0.015712] [time: 173.970990]
[Epoch 19541/20000] [loss: 0.015927] [time: 173.978981]
[Epoch 19542/20000] [loss: 0.015547] [time: 173.

[Epoch 19680/20000] [loss: 0.015996] [time: 175.219491]
[Epoch 19681/20000] [loss: 0.015931] [time: 175.228450]
[Epoch 19682/20000] [loss: 0.015660] [time: 175.237414]
[Epoch 19683/20000] [loss: 0.013940] [time: 175.246373]
[Epoch 19684/20000] [loss: 0.015539] [time: 175.255332]
[Epoch 19685/20000] [loss: 0.014987] [time: 175.263298]
[Epoch 19686/20000] [loss: 0.013636] [time: 175.272264]
[Epoch 19687/20000] [loss: 0.015426] [time: 175.280222]
[Epoch 19688/20000] [loss: 0.014170] [time: 175.289183]
[Epoch 19689/20000] [loss: 0.015835] [time: 175.297149]
[Epoch 19690/20000] [loss: 0.016329] [time: 175.305112]
[Epoch 19691/20000] [loss: 0.015211] [time: 175.314076]
[Epoch 19692/20000] [loss: 0.014390] [time: 175.322039]
[Epoch 19693/20000] [loss: 0.016387] [time: 175.330004]
[Epoch 19694/20000] [loss: 0.017341] [time: 175.337976]
[Epoch 19695/20000] [loss: 0.016279] [time: 175.346934]
[Epoch 19696/20000] [loss: 0.013935] [time: 175.355890]
[Epoch 19697/20000] [loss: 0.015737] [time: 175.

[Epoch 19834/20000] [loss: 0.015095] [time: 176.579498]
[Epoch 19835/20000] [loss: 0.016501] [time: 176.587413]
[Epoch 19836/20000] [loss: 0.017996] [time: 176.596374]
[Epoch 19837/20000] [loss: 0.015534] [time: 176.604343]
[Epoch 19838/20000] [loss: 0.014780] [time: 176.612356]
[Epoch 19839/20000] [loss: 0.014460] [time: 176.621314]
[Epoch 19840/20000] [loss: 0.014296] [time: 176.629279]
[Epoch 19841/20000] [loss: 0.014673] [time: 176.638186]
[Epoch 19842/20000] [loss: 0.017805] [time: 176.646150]
[Epoch 19843/20000] [loss: 0.014939] [time: 176.655110]
[Epoch 19844/20000] [loss: 0.015708] [time: 176.664070]
[Epoch 19845/20000] [loss: 0.014805] [time: 176.673031]
[Epoch 19846/20000] [loss: 0.015452] [time: 176.680996]
[Epoch 19847/20000] [loss: 0.016168] [time: 176.688961]
[Epoch 19848/20000] [loss: 0.014412] [time: 176.697922]
[Epoch 19849/20000] [loss: 0.016599] [time: 176.705889]
[Epoch 19850/20000] [loss: 0.017224] [time: 176.714849]
[Epoch 19851/20000] [loss: 0.014694] [time: 176.

[Epoch 19983/20000] [loss: 0.015513] [time: 177.920588]
[Epoch 19984/20000] [loss: 0.016100] [time: 177.929550]
[Epoch 19985/20000] [loss: 0.015630] [time: 177.938511]
[Epoch 19986/20000] [loss: 0.015687] [time: 177.947111]
[Epoch 19987/20000] [loss: 0.016212] [time: 177.956072]
[Epoch 19988/20000] [loss: 0.015476] [time: 177.965044]
[Epoch 19989/20000] [loss: 0.015785] [time: 177.973994]
[Epoch 19990/20000] [loss: 0.014826] [time: 177.981957]
[Epoch 19991/20000] [loss: 0.015647] [time: 177.990918]
[Epoch 19992/20000] [loss: 0.014785] [time: 177.999827]
[Epoch 19993/20000] [loss: 0.016814] [time: 178.007792]
[Epoch 19994/20000] [loss: 0.016047] [time: 178.015757]
[Epoch 19995/20000] [loss: 0.014260] [time: 178.024768]
[Epoch 19996/20000] [loss: 0.016395] [time: 178.032732]
[Epoch 19997/20000] [loss: 0.013473] [time: 178.040699]
[Epoch 19998/20000] [loss: 0.016407] [time: 178.049665]
[Epoch 19999/20000] [loss: 0.014532] [time: 178.068575]


In [ ]:
loss_training = loss_training.detach().numpy()
loss_test = loss_test.detach().numpy()

In [20]:
DataFrame(loss_training).to_csv('Saved results/loss_training_mfd.csv',index=False)
DataFrame(loss_test).to_csv('Saved results/loss_test_mfd.csv',index=False)

In [14]:
v_estimation_pred,v_f,k_c, mm = mfd(training_set_k[0])

In [15]:
v_f = v_f.detach().numpy()
k_c = k_c.detach().numpy()
mm = mm.detach().numpy()

In [17]:
DataFrame(v_f).to_csv('parameter/v_f.csv',index=False)
DataFrame(k_c).to_csv('parameter/k_c.csv',index=False)
DataFrame(mm).to_csv('parameter/mm.csv',index=False)